In [1]:
# 1. Establish a connection between Python and the Sakila database.
import mysql.connector
from getpass import getpass

In [4]:
password = getpass()

cnx = mysql.connector.connect(user = 'root',
                              password = password,
                              host = 'localhost',
                              database = 'sakila')

In [5]:
# cheching if it is connected
cnx.is_connected()

True

In [46]:
# let's define an object that we will use to interact with the database
cursor= cnx.cursor()

In [48]:
# 2. Write a Python function called rentals_month that retrieves rental data for a given month and year
import pandas as pd

def rentals_month (month, year):
    query = (f"""SELECT rental_date, inventory_id, customer_id, return_date, staff_id, last_update
             FROM sakila.rental
             WHERE MONTH(rental_date) = {month} and YEAR(rental_date) = {year};""")
    #cursor = cnx.cursor()
    cursor.execute(query)
    results = pd.DataFrame(cursor.fetchall())
    df = pd.DataFrame(results)
    df.columns = [head[0] for head in cursor.description]
    return df

In [55]:
df = rentals_month(month = 5,year = 2005)
df

,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...
1151,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [58]:
# 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year 
# and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

def rental_count_month (month, year):
    df_rental = rentals_month(month = month,year = year)
    df_rental = df_rental[(df_rental['rental_date'].dt.month == month) & (df_rental['rental_date'].dt.year == year) ]
    df_counts = df_rental.groupby('customer_id').size().reset_index(name=f'rentals_{month}_{year}')
    return df_counts

In [61]:
rental_count_month(month=5,year=2005)

,customer_id,rentals_5_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [68]:
# 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in 
# different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

def compare_rentals(month1,year1,month2,year2):
    df1 = rental_count_month(month=month1,year=year1)
    df2 = rental_count_month(month=month2,year=year2)
    merged_df = pd.merge(df1, df2, on = 'customer_id', how='outer')
    merged_df['difference'] = merged_df.iloc[:, 1]  - merged_df.iloc[:, 2] 
    return merged_df

In [69]:
compare_rentals(month1=5,year1=2005,month2=6,year2=2005)

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,5,3.0,5.0,-2.0
4,6,3.0,4.0,-1.0
...,...,...,...,...
593,583,NaN,6.0,NaN
594,585,NaN,4.0,NaN
595,591,NaN,3.0,NaN
596,592,NaN,5.0,NaN
